In [27]:
import requests
import pandas as pd
import json
from dateutil import parser

In [28]:
API_KEY = "667e94661abb533b7c59e065eed32b7a-e8470dd11ead1617b105cb8843647d6c"
ACCOUNT_ID = "101-001-25318071-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [29]:
session = requests.Session()

In [30]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [31]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [32]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [33]:
response = session.get(url, params=None, data=None, headers=None)

In [34]:
response.status_code

200

In [35]:
data = response.json()

In [36]:
instruments_list = data['instruments']

In [37]:
len(instruments_list)

68

In [38]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [39]:
key_inst = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [40]:
instruments_dict = {}

for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_inst}

In [41]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [42]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [43]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
        )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()

    if response.status_code == 200:
        if "candles" not in data:
            data = []
        else: 
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ["mid", "bid", "ask"]
    ohlc = ["o", "h", "l", "c"]

    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']

        for p in prices:
            for o in ohlc:
                new_dict[f'{p}_{o}'] = float(candle[p][o])
        
        final_data.append(new_dict)

    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    
    if len(data) == 0:
        print("no candles", pair_name)

    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")


In [44]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [46]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 10 candles, 2023-08-10 05:00:00+00:00 2023-08-11 17:00:00+00:00
